In [1]:
import os
from pyspark import SparkConf, SparkContext

os.environ['PYSPARK_PYTHON']='/home/hadoop/anaconda3/envs/word/bin/python'

import jieba
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pandas as pd
from wordcloud import WordCloud
import numpy as np
from PIL import Image
from pyecharts.render import make_snapshot
from snapshot_selenium import snapshot
from pyecharts import options as opts
from pyecharts.charts import Pie, Bar, Funnel, Scatter

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

import torch
from wordsegment import WordSegmenter
from transformers import AutoTokenizer, AutoModelForTokenClassification

/home/hadoop/anaconda3/envs/word/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import sys
pythonpath = sys.executable
print(pythonpath)

/home/hadoop/anaconda3/envs/word/bin/python


: 

In [ ]:
# 配置spark
  
conf = SparkConf().setMaster("local").setAppName("lab2")

sc = SparkContext(conf=conf)
sc.stop()
sc = SparkContext(conf=conf)

# 读取停用词文件
stopWords_filePath = "src/stop_words.txt"
with open(stopWords_filePath, "r", encoding="utf-8") as file:
    stopWords = [line.strip() for line in file.readlines()]

# 读取文本内容并进行分词
data_filePath = "src/data.txt"
dataRdd = sc.textFile(data_filePath)
text = dataRdd.reduce(lambda a, b: a + " " + b)
lines_list = text.splitlines()

#print(lines_list[:1])
print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
#words_list = jieba.lcut(text)

#bert分词
tokenizer = AutoTokenizer.from_pretrained("ckiplab/albert-base-chinese-ws")

model = AutoModelForTokenClassification.from_pretrained("ckiplab/albert-base-chinese-ws")
ws = WordSegmenter(model=model,tokenizer=tokenizer,device = torch.device("cuda" if torch.cuda.is_available() else "cpu"))
words_list = ws.segment(lines_list)
print(len(ws.segment(lines_list)))

print(words_list[:1])



!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [ ]:
import itertools
# 扁平化 words_list
flat_words_list = list(itertools.chain(*words_list))

In [ ]:
# 创建RDD并进行词频统计
#wordsRdd = sc.parallelize(words_list)
wordsRdd = sc.parallelize(flat_words_list)
print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
#print(wordsRdd[:1])
resRdd = wordsRdd.filter(lambda word: word not in stopWords) \
    .filter(lambda word: len(word) > 1) \
        .map(lambda word: (word, 1)) \
            .reduceByKey(lambda a, b: a + b) \
                .sortBy(lambda x: x[1], ascending=False)

In [ ]:
# 设置中文字体
font_name = "SimHei"
plt.rcParams['font.family']= font_name # 指定字体，实际上相当于修改 matplotlibrc 文件　只不过这样做是暂时的　下次失效
plt.rcParams['axes.unicode_minus']=False # 正确显示负号，防止变成方框

print(type(resRdd))
print(words_list[:])
# 绘制词频直方图，展示前20个词语的词频分布
top20 = resRdd.take(20)
words, frequencies = zip(*top20)
plt.figure(figsize=(12, 8))
sns.barplot(y=list(words), x=list(frequencies))
plt.title("词频排名前二十", fontsize=15)
plt.xlabel("频率", fontsize=15)
plt.ylabel("词语", fontsize=15)
plt.show()

In [ ]:
# 词云生成
top50 = resRdd.take(50)
wc = WordCloud(font_path="/home/hadoop/anaconda3/envs/word/lib/python3.9/site-packages/matplotlib/mpl-data/fonts/ttf/SimHei.ttf",
               background_color="white", 
               width=1920, height=1080, 
               max_words=2000, 
               margin=5)
wc.generate_from_frequencies(dict(top50))
plt.figure(figsize=(12, 8))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
# 将RDD转换为Dic，并截取指定长度topK
def rdd2dic(resRdd,topK):
        """
        将RDD转换为Dic，并截取指定长度topK
        :param resRdd: 词频统计降序排序结果RDD
        :param topK: 截取的指定长度
        :return:
        """
        # 提示：SparkRdd有函数可直接转换
        resDic = resRdd.collectAsMap()
        # 截取字典前K个
        K = 0
        wordDicK = {}
        for key, value in resDic.items():
            # 完成循环截取字典
            if K >= topK:
                break
            wordDicK[key] = value
            K += 1
        return wordDicK

In [ ]:
SAVAPATH = '/home/hadoop/ex2/Lab2_WordCount/results_bert/'

In [ ]:
# 生成词云
wwDic = rdd2dic(resRdd,50)
wc = WordCloud(font_path='/home/hadoop/anaconda3/envs/word/lib/python3.9/site-packages/matplotlib/mpl-data/fonts/ttf/SimHei.ttf',
                       background_color='white',
                       max_words=2000,
                       width=1920, height=1080,
                       margin=5)
wc.generate_from_frequencies(wwDic)
plt.figure(figsize=(12, 8))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()
if not os.path.exists(SAVAPATH):
            os.makedirs(SAVAPATH)
wc.to_file(os.path.join(SAVAPATH, '词云可视化.png'))

In [ ]:
import matplotlib.image as mpimg

在此之前需要安装“WenQuanYi Zen Hei”字体，命令：sudo apt-get install fonts-wqy-zenhei（ubuntu）

In [ ]:
# 饼图可视化

pieDic = rdd2dic(resRdd,10)
key_list = pieDic.keys()      # wordDic所有key组成list
value_list= pieDic.values()   # wordDic所有value组成list
def pie_position() -> Pie:
    c = (
        Pie()
            .add
            (
            "",
            [list(z) for z in zip(key_list, value_list)], # dic -> list
            center=["35%", "50%"],
            )
            .set_global_opts
            (
            title_opts=opts.TitleOpts(
                title='饼图可视化',
                title_textstyle_opts=opts.TextStyleOpts(font_size=18, font_family="WenQuanYi Zen Hei", color="#333")
                ), # 设置标题
            legend_opts=opts.LegendOpts(
                pos_left="15%",
                textstyle_opts=opts.TextStyleOpts(font_size=12, color="#666")  # 设置图例字体
                ),
            )
            .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
    )
    return c

image_path = SAVAPATH + '饼图可视化.png'

# 保存结果
make_snapshot(snapshot, pie_position().render(), image_path)
img = mpimg.imread(image_path)
plt.figure(figsize=(8, 6))
plt.imshow(img)
plt.axis('off')  # 不显示坐标轴
plt.show()

In [ ]:
# 柱状图可视化
Dic = rdd2dic(resRdd,15)

key_list = Dic.keys()      # wordDic所有key组成list
value_list= Dic.values()   # wordDic所有value组成list
def bar_position() -> Bar:
    bar = (
        Bar()
            .add_xaxis(list(key_list))
            .add_yaxis("次数", list(value_list))
            .set_global_opts
            (
            title_opts=opts.TitleOpts(title='柱状图可视化'), # 设置标题
            legend_opts=opts.LegendOpts(pos_left="15%"),
            )
            .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
        )
    return bar
# 保存结果

image_path = SAVAPATH + '柱状图可视化.png'
make_snapshot(snapshot, bar_position().render(), image_path)

img = mpimg.imread(image_path)
plt.figure(figsize=(8, 6))
plt.imshow(img)
plt.axis('off')  # 不显示坐标轴
plt.show()

In [ ]:
# 漏斗图可视化
Dic = rdd2dic(resRdd,15)

key_list = list(Dic.keys())      # wordDic所有key组成list
value_list= list(Dic.values())   # wordDic所有value组成list
data = [[key_list[i],value_list[i]]for i in range(len(key_list))]
def funnel_position() -> Funnel:
    funnel = (
        Funnel()
            .add(
                series_name="",
                data_pair=data,
                gap=2,
                tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{a} <br/>{b} : {c}%"),
                label_opts=opts.LabelOpts(is_show=True, position="inside"),
                itemstyle_opts=opts.ItemStyleOpts(border_color="#fff", border_width=1),
            )
            .set_global_opts(title_opts=opts.TitleOpts(title="漏斗图"))
    )
    return funnel
# 保存结果

image_path = SAVAPATH + '漏斗图可视化.png'
make_snapshot(snapshot, funnel_position().render(), image_path)

img = mpimg.imread(image_path)
plt.figure(figsize=(8, 6))
plt.imshow(img)
plt.axis('off')  # 不显示坐标轴
plt.show()

In [ ]:
# 散点图可视化

Dic = rdd2dic(resRdd,15)

key_list = list(Dic.keys())      # wordDic所有key组成list
value_list= list(Dic.values())   # wordDic所有value组成list
def scatter_position() -> Funnel:
    scatter = (
        Scatter()
            .add_xaxis(key_list)
            .add_yaxis("次数", value_list)
            .set_global_opts(
                title_opts=opts.TitleOpts(title="Scatter-VisualMap(Size)"),
                visualmap_opts=opts.VisualMapOpts(type_="size", max_=150, min_=20),
        )
    )
    return scatter
# 保存结果
image_path = SAVAPATH + '散点图可视化.png'
make_snapshot(snapshot, scatter_position().render(), image_path)

img = mpimg.imread(image_path)
plt.figure(figsize=(8, 6))
plt.imshow(img)
plt.axis('off')  # 不显示坐标轴
plt.show()